In [241]:
import pandas as pd
import numpy as np

In [242]:
path = 'Example1/'
#path = 'Example2/'

In [243]:
contribution_capacity = 2


weight_coefficient = 0.8 # determines how much weight we should give to the node itself and the rest to the sum of the neighbors
penalty = 100 #we penalize the score of those nodes that don't have minimum skill weight by this much

In [244]:
def load_network_data(path):
    network = pd.read_csv(path+'network_topology.csv')
    skills = pd.read_csv(path+'skill_sets.csv')
    requirements = pd.read_csv(path+'requirements.csv')
    
    return network, skills, requirements

In [245]:
def neighbor_skill_sum(row):
    global network,skill_sets
    n1 = network[(network.node1 == row.node) | (network.node2 == row.node)]
    neighbors = pd.concat([n1.node1,n1.node2]).unique().tolist().copy()
    neighbors.remove(row.node)
    neighbors_skill_sum = sum(skill_sets[(skill_sets.node.isin(neighbors)) & (skill_sets.skill == row.skill)].weight)
    return neighbors_skill_sum

In [246]:
def calculate_score(row, weight_coef, penalty):
    print(neighbor_skill_sum(row))
    score = weight_coef*row['weight']+((1-weight_coef)*neighbor_skill_sum(row))
    if row['weight']<row['min_weight']:
        score = score - penalty
    
    return score


In [247]:
network, skill_sets, requirements = load_network_data(path)

In [248]:
requirements

,skill,number_needed,min_weight
0,s1,2,5
1,s3,1,4
2,s4,2,4


In [249]:
network

,node1,node2
0,1,2
1,1,3
2,1,4
3,2,3
4,2,4
5,3,4


In [250]:
skill_sets

,node,skill,weight
0,1,s1,10
1,2,s1,5
2,3,s2,6
3,3,s4,7
4,4,s2,10
5,4,s3,7
6,4,s4,10


In [251]:
skill_sets_merged = skill_sets.merge(requirements,on = 'skill')

In [252]:
skill_sets_merged

,node,skill,weight,number_needed,min_weight
0,1,s1,10,2,5
1,2,s1,5,2,5
2,3,s4,7,2,4
3,4,s4,10,2,4
4,4,s3,7,1,4


In [253]:
skill_sets_merged['min_weight'] = skill_sets_merged['min_weight'].fillna(0) # filling un-needed skills with minimum weight of zero

In [254]:
skill_sets_merged

,node,skill,weight,number_needed,min_weight
0,1,s1,10,2,5
1,2,s1,5,2,5
2,3,s4,7,2,4
3,4,s4,10,2,4
4,4,s3,7,1,4


In [255]:
skill_sets_merged['score'] = skill_sets_merged.apply(calculate_score, args=(weight_coefficient, penalty), axis=1) # calculate score for each row of skill_set_merged
skill_sets_merged['selected'] = 0 # setting initial selected flag as zero

5
10
10
7
0


In [256]:

skill_sets_merged

,node,skill,weight,number_needed,min_weight,score,selected
0,1,s1,10,2,5,9.0,0
1,2,s1,5,2,5,6.0,0
2,3,s4,7,2,4,7.6,0
3,4,s4,10,2,4,9.4,0
4,4,s3,7,1,4,5.6,0


In [257]:
# creating the list of unique nodes
unique_nodes = list(set(skill_sets_merged.node))
unique_nodes 

[1, 2, 3, 4]

In [258]:
#defining a dataframe with default contribution capacity
capacity = pd.DataFrame(unique_nodes,columns=['node'])
capacity['cap'] = contribution_capacity

In [259]:
capacity

,node,cap
0,1,2
1,2,2
2,3,2
3,4,2


In [260]:
requirements.skill

0    s1
1    s3
2    s4
Name: skill, dtype: object

In [261]:
skill_sets_merged.skill

0    s1
1    s1
2    s4
3    s4
4    s3
Name: skill, dtype: object

In [262]:
requirements.number_needed

0    2
1    1
2    2
Name: number_needed, dtype: int64

In [263]:

for s,number in zip(list(requirements.skill),list(requirements.number_needed)):
    selected_group = skill_sets_merged[(skill_sets_merged.skill == s) ]
    selected_group = selected_group.merge(capacity,on='node')
    selected_group = selected_group[selected_group.cap>0].sort_values(by='score',ascending=False).head(number)
    capacity.loc[capacity.node.isin(selected_group.node),'cap'] =  capacity.loc[capacity.node.isin(selected_group.node),'cap'] - 1
    skill_sets_merged['selected'] = ((skill_sets_merged['selected'] == 1) | ((skill_sets_merged.node.isin(selected_group.node)) & (skill_sets_merged.skill == s))).astype(int)

In [264]:
final_results = skill_sets_merged[skill_sets_merged.selected == 1].sort_values(by=['skill','score'],ascending=[True,False]).reset_index(drop=True)

In [265]:
final_results

,node,skill,weight,number_needed,min_weight,score,selected
0,1,s1,10,2,5,9.0,1
1,2,s1,5,2,5,6.0,1
2,4,s3,7,1,4,5.6,1
3,4,s4,10,2,4,9.4,1
4,3,s4,7,2,4,7.6,1
